In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from gradient_descent import Objective, gradient_descent_momentum

In [2]:
df_train = pd.read_csv('./task1a/train.csv')
x=df_train.iloc[:,1:].to_numpy()
y=df_train.iloc[:,0].to_numpy()

egv,evec=np.linalg.eig(np.transpose(x)@x)
eta=1/max(egv)

In [3]:
class ToyObjective(Objective):
    def __init__(self, x, y, lda):
        self.x = x
        self.y = y
        self.lda = lda
        
    def __call__(self, w):
        xw=self.x@w
        return np.transpose(xw)@xw-2*np.transpose(xw)@y+np.transpose(y)@y + self.lda*np.transpose(w)@w
        #return np.sum(np.square((self.y - self.x@w))) + self.lda*np.sum(np.square(w))
    
    
    def grad(self, w):
        return  2*np.sum(-np.transpose(np.transpose(self.x)*(self.y - self.x@w)), axis=0) + 2*lda*w


In [4]:
lda=0.1
obj = ToyObjective(x,y,lda)
w = np.ones(13)
obj.grad(w)

array([1.36453442e+06, 2.60013378e+06, 3.25741215e+06, 2.05640834e+04,
       1.51695525e+05, 1.66054856e+06, 1.89186640e+07, 9.86574602e+05,
       3.06095032e+06, 1.21525881e+08, 4.99641194e+06, 9.32659237e+07,
       3.77396939e+06])

In [5]:
learning_rate = 0.9*eta
tol = 1e-10
n_steps = 100000
w_init = w
normalize = False

In [6]:
results = gradient_descent_momentum(obj, w_init, learning_rate=learning_rate, tol=tol, n_steps=n_steps, normalize=normalize)
results[2]

array([1.24918764e+08, 7.30894673e+08, 1.62852982e+08, ...,
       3.83259316e+03, 3.83259287e+03, 3.83259259e+03])

In [7]:
w_opt=results[0]
w_opt

array([-4.28938293e-02,  4.18525890e-02, -3.67907429e-04,  1.92102190e+00,
        1.04606022e+00,  5.03484713e+00,  1.54380322e-02, -7.19715963e-01,
        3.79089620e-01, -1.35815011e-02, -3.05515078e-01,  2.14489609e-02,
       -6.41289939e-01])

In [8]:
np.sum(np.square((y - x@w_opt)))

3829.4620835590963

In [9]:
obj.grad(w_opt)

array([-1.88607595e-01,  2.46131641e-01,  8.18189194e-01, -2.80727977e+01,
        6.11300607e+00, -2.49610673e+01,  4.95356026e-01,  5.85294956e+00,
       -3.49964704e-01,  5.03370667e-03,  5.85402320e+00,  1.64170247e-02,
       -1.95838869e+00])

In [12]:

n_fold=10
len_fold=y.shape[0]//n_fold
k=9
k_fold=np.concatenate((y[:k*len_fold], y[(k+1)*len_fold:]))
x_k=np.concatenate((x[:k*len_fold], x[(k+1)*len_fold:]))
x_k.shape

(135, 13)

In [13]:
lda_list=[0.1, 1, 10, 100, 200]
for lda in lda_list:
    print(lda)

0.1
1
10
100
200


In [14]:
xT=np.transpose(x)

In [15]:
w = np.linalg.inv(xT @ x) @ xT @ y
w

array([-0.03833324,  0.03733255,  0.02237796,  3.71989865, -5.6979845 ,
        5.35579469,  0.01440385, -0.79532608,  0.35229042, -0.01114316,
       -0.29588   ,  0.02179952, -0.59271428])

In [16]:
y_pred = x @ w_opt
np.sum(np.square((y - y_pred)))

3829.4620835590963

In [17]:
RMSE = mean_squared_error(y, y_pred)**0.5
RMSE

5.052697024731179

In [18]:
RMSE=np.arange(12).reshape((3, 4))
RMSE

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [19]:
mRM=np.mean(RMSE,axis=1)

In [20]:
pd.DataFrame(mRM).to_csv("nRM.csv", header=None, index=None)

In [26]:
RMSE=np.zeros((len(lda_list),n_fold,x.shape[1]))
RMSE[0,1,:]=w
RMSE

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [-0.03833324,  0.03733255,  0.02237796,  3.71989865,
         -5.6979845 ,  5.35579469,  0.01440385, -0.79532608,
          0.35229042, -0.01114316, -0.29588   ,  0.02179952,
         -0.59271428],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.   

In [29]:
for i in range(len(lda_list)):
    pd.DataFrame(RMSE[i]).to_csv(f'./task1a/RMSE_test{i}.csv', header=None, index=None)

In [40]:
try:
    df_w = pd.read_csv('./task1a/w_opt_lda=1.csv', header=None)
    w = df_w.to_numpy()
except:
    w = np.ones(()) 
w

array([[-2.95275856e-02,  4.01350168e-02, -1.86449223e-02,
         1.87540836e+00,  1.10555783e+00,  4.89300038e+00,
         4.73413617e-03, -6.71651964e-01,  3.87243218e-01,
        -1.65187177e-02, -2.15514929e-01,  2.14206406e-02,
        -5.66393465e-01],
       [-3.30976017e-02,  3.70495650e-02,  7.08740935e-03,
         1.92498578e+00,  9.47328376e-01,  5.39912398e+00,
         7.60628707e-04, -7.81281241e-01,  3.79404558e-01,
        -1.42038241e-02, -4.26447909e-01,  2.36187593e-02,
        -5.91392076e-01],
       [-2.96562605e-02,  2.63933251e-02, -7.81621759e-02,
         1.40810466e+00,  9.59728489e-01,  5.18365574e+00,
         2.17756587e-02, -6.02051655e-01,  3.49764254e-01,
        -9.96661591e-03, -4.67027112e-01,  2.28378373e-02,
        -5.85285286e-01],
       [-3.54306073e-02,  5.26927014e-02, -1.60562724e-02,
         2.07671287e+00,  1.00311565e+00,  4.90755448e+00,
         3.41334753e-02, -7.69047112e-01,  3.76156735e-01,
        -1.32390113e-02, -2.78844805e